In [ ]:
# CHANGE TABLE COLUMNS TO GOOD ONES FOR PGR

In [1]:
import sqlalchemy as sql
from sqlalchemy_utils import database_exists
import psycopg2
from geoalchemy2 import WKTElement, Geometry

In [2]:
hostname = "wheelway2.cgfv5tiyps6x.us-east-1.rds.amazonaws.com"
username = "postgres"
with open('/home/adam/rdskey') as keyfile:
    rds_key = keyfile.readline().strip()

    


In [3]:
con = psycopg2.connect(database = "wheelway", 
                       user=username, 
                       host=hostname, 
                       password=rds_key, 
                       port=5432
                      )

In [4]:
engine = sql.create_engine('postgresql+psycopg2://{user}:{pwd}@{host}:{port}/wheelway'.format(user=username,
                                                                                     pwd=rds_key,
                                                                                     host=hostname,
                                                                                     port=5432
                                                                                     ))
con = engine.connect()

In [5]:
import pickle as pkl
import networkx as nx

with open("../brighton_graph.pkl", 'rb') as pklfile:
    brighton_G = pkl.load(pklfile)

In [6]:
import osmnx as ox 
_, gdf_edges = ox.graph_to_gdfs(brighton_G)

In [7]:
#gdf_points['geom'] = gdf_points['geometry'].apply(lambda x: WKTElement(x.wkt, srid=4326))
gdf_edges['geom'] = gdf_edges['geometry'].apply(lambda x: WKTElement(x.wkt, srid=4326))
gdf_edges.drop(columns=['geometry'], inplace=True)

In [8]:
gdf_edges.rename(columns = {'u':'source',
                 'v':'target',
                 'length_m':'cost'}, inplace=True)

In [14]:
gdf_edges['osmid'] = gdf_edges['osmid'].map(lambda x : x if x < 1000000000 else x / 1000000 )

In [15]:
gdf_edges['osmid'].describe()

count    2.110460e+05
mean     9.946978e+07
std      1.592797e+08
min      0.000000e+00
25%      3.520525e+07
50%      6.198700e+07
75%      8.881875e+07
max      9.999000e+08
Name: osmid, dtype: float64

In [11]:
ALPHA = 2/5
gdf_edges['balanced_l'] = gdf_edges['cost'] * (1 + ALPHA * abs(gdf_edges['angle_deg'])/15)

In [ ]:
len(gdf_edges['osmid'].unique())

In [16]:
from sqlalchemy import Integer, Float, BigInteger
gdf_edges.to_sql('my_edges', engine, if_exists='replace', index=False, method='multi', chunksize=1000,
                 dtype={'geom': Geometry('LINESTRING', srid=4326),
                                'angle_class': Integer,
                                'balanced_l': Float,
                                'cost': Float,
                                'osmid': Integer,
                                'source': Integer,
                                'target': Integer})